In [ ]:
!pip install scikit-plot
!pip install jax-unirep

Try to deal with GPU memory 
accroading to https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html

In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

from Bio import SeqIO
from jax_unirep import get_reps
from jax_unirep import evotune, fit
from jax_unirep.utils import dump_params

/home/kongkitimanonk/.conda/envs/jupyter_NB/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [2]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='False'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']='.70'

os.environ['MKL_NUM_THREADS']='1' 
os.environ['OPENBLAS_NUM_THREADS']='1'

os.environ["NUM_INTER_THREADS"]="1"
os.environ["NUM_INTRA_THREADS"]="1"
os.environ.update(
        XLA_FLAGS=(
            '--xla_cpu_multi_thread_eigen=false '
            'intra_op_parallelism_threads=2 '
            'inter_op_parallelism_threads=2 '
            'OMP_NUM_THREADS=1  '
        ),
        XLA_PYTHON_CLIENT_PREALLOCATE='false',
    )

In [3]:
fastas = "/home/kongkitimanonk/SCRATCH_NOBAK/cd-hit/bk_fasta/SRR5868581_2.len20.assembly.fasta"
plk = "/home/kongkitimanonk/SCRATCH_NOBAK/phase3/TaiwaneseOolong.len20.pkl"

In [4]:
with open(fastas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    seqs = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        # Remove leading and trailing characters from a string
        seqs.append(str(seq_record.seq.strip('*')))
        lengths.append(len(seq_record.seq))

In [5]:
# dictionary of lists  
dict = {'ID': identifiers, 'Sequence': seqs, 'length': lengths}  
df = pd.DataFrame(dict) 
#df["Sequence"] =  seqs
#df.to_pickle(plk)    
df  

,ID,Sequence,length
0,0,GRWNNAGKSEEHTSELQSRSDLVCRLLLEKKRGGGG,36
1,1,MHIGKIEEHTSELQSRSDLVCRHQLEKKNGGG,32
2,2,DVCSSDYPGFRFIPHRQVCLPKMAHLELSIPWRGSTEQPRRP,42
3,3,TTLGQTEEHTSELQSRSDLVCRLQLEKKKGG,31
4,4,YDGYPSSNTFKLTLTDLMSHSQFHSLN,28
...,...,...,...
1058124,1058124,AGHTEQIGRAHWARSEEHTGQDRKRTQ,28
1058125,1058125,SEEHTSELQSRSYLVSRLLLEKKKEG,26
1058126,1058126,SSLIASSAYQKWHTWSSRFRGAAQRSSRAVLPI,34
1058127,1058127,ASPVLLTKNGTLGALDSVARLNGAAAPSYLFKV,34


Remove Duplicate

In [6]:
df.drop_duplicates(subset=['Sequence'],inplace=True)
df

,ID,Sequence,length
0,0,GRWNNAGKSEEHTSELQSRSDLVCRLLLEKKRGGGG,36
1,1,MHIGKIEEHTSELQSRSDLVCRHQLEKKNGGG,32
2,2,DVCSSDYPGFRFIPHRQVCLPKMAHLELSIPWRGSTEQPRRP,42
3,3,TTLGQTEEHTSELQSRSDLVCRLQLEKKKGG,31
4,4,YDGYPSSNTFKLTLTDLMSHSQFHSLN,28
...,...,...,...
1058124,1058124,AGHTEQIGRAHWARSEEHTGQDRKRTQ,28
1058125,1058125,SEEHTSELQSRSYLVSRLLLEKKKEG,26
1058126,1058126,SSLIASSAYQKWHTWSSRFRGAAQRSSRAVLPI,34
1058127,1058127,ASPVLLTKNGTLGALDSVARLNGAAAPSYLFKV,34


In [ ]:
#0-579590
#579590-1159181

df1 = datasX.iloc[:, :]
df2 = datasX.iloc[:, :]


In [8]:
_h_avg, h_final, c_final= get_reps(df['Sequence'].to_list())
df['reps']=_h_avg.tolist()
df

KeyboardInterrupt: 

In [ ]:
df.to_pickle(plk)
#tmp_df = pd.read_pickle(plk)
#tmp_df